Курсовой проект
Постановка задачи
Задача

Требуется, на основании имеющихся данных о клиентах банка, построить модель, используя обучающий датасет, для прогнозирования невыполнения долговых обязательств по текущему кредиту. Выполнить прогноз для примеров из тестового датасета.

Наименование файлов с данными

course_project_train.csv - обучающий датасет
course_project_test.csv - тестовый датасет

Целевая переменная

Credit Default - факт невыполнения кредитных обязательств

Метрика качества

F1-score (sklearn.metrics.f1_score)

Описание датасета

Home Ownership - домовладение
Annual Income - годовой доход
Years in current job - количество лет на текущем месте работы
Tax Liens - налоговые обременения
Number of Open Accounts - количество открытых счетов
Years of Credit History - количество лет кредитной истории
Maximum Open Credit - наибольший открытый кредит
Number of Credit Problems - количество проблем с кредитом
Months since last delinquent - количество месяцев с последней просрочки платежа
Bankruptcies - банкротства
Purpose - цель кредита
Term - срок кредита
Current Loan Amount - текущая сумма кредита
Current Credit Balance - текущий кредитный баланс
Monthly Debt - ежемесячный долг
Credit Default - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [1]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2, f1_score as f1, precision_score as prec, recall_score as recall, classification_report as report
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
import xgboost as xgb
import catboost as cat
from catboost import CatBoostClassifier


from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '%.5f' % x)
df_1 = pd.read_csv('Downloads\course_project_train.csv')
df_2 = pd.read_csv('Downloads\course_project_test.csv')
df_1.describe()

,Annual Income,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
count,5943.00000,7500.00000,7500.00000,7500.00000,7500.00000,7500.00000,3419.00000,7486.00000,7500.00000,7500.00000,7500.00000,5943.00000,7500.00000
mean,1366391.72017,0.03013,11.13093,18.31747,945153.72747,0.17000,34.69260,0.11715,11873177.44507,289833.23520,18314.45413,1151.08750,0.28173
std,845339.19963,0.27160,4.90892,7.04195,16026216.67085,0.49860,21.68881,0.34719,31926122.97473,317871.38238,11926.76467,1604.45142,0.44987
min,164597.00000,0.00000,2.00000,4.00000,0.00000,0.00000,0.00000,0.00000,11242.00000,0.00000,0.00000,585.00000,0.00000
25%,844341.00000,0.00000,8.00000,13.50000,279229.50000,0.00000,16.00000,0.00000,180169.00000,114256.50000,10067.50000,711.00000,0.00000
50%,1168386.00000,0.00000,10.00000,17.00000,478159.00000,0.00000,32.00000,0.00000,309573.00000,209323.00000,16076.50000,731.00000,0.00000
75%,1640137.00000,0.00000,14.00000,21.80000,793501.50000,0.00000,50.00000,0.00000,519882.00000,360406.25000,23818.00000,743.00000,1.00000
max,10149344.00000,7.00000,43.00000,57.70000,1304726170.00000,7.00000,118.00000,4.00000,99999999.00000,6506797.00000,136679.00000,7510.00000,1.00000


In [1403]:
df_1.keys()

Index(['Home Ownership', 'Annual Income', 'Years in current job', 'Tax Liens',
       'Number of Open Accounts', 'Years of Credit History',
       'Maximum Open Credit', 'Number of Credit Problems',
       'Months since last delinquent', 'Bankruptcies', 'Purpose', 'Term',
       'Current Loan Amount', 'Current Credit Balance', 'Monthly Debt',
       'Credit Score', 'Credit Default'],
      dtype='object')

In [1404]:
df_1.dtypes

Home Ownership                   object
Annual Income                   float64
Years in current job             object
Tax Liens                       float64
Number of Open Accounts         float64
Years of Credit History         float64
Maximum Open Credit             float64
Number of Credit Problems       float64
Months since last delinquent    float64
Bankruptcies                    float64
Purpose                          object
Term                             object
Current Loan Amount             float64
Current Credit Balance          float64
Monthly Debt                    float64
Credit Score                    float64
Credit Default                    int64
dtype: object

In [1405]:
df_1.head()

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.00000,NaN,0.00000,11.00000,26.30000,685960.00000,1.00000,nan,1.00000,debt consolidation,Short Term,99999999.00000,47386.00000,7914.00000,749.00000,0
1,Own Home,1025487.00000,10+ years,0.00000,15.00000,15.30000,1181730.00000,0.00000,nan,0.00000,debt consolidation,Long Term,264968.00000,394972.00000,18373.00000,737.00000,1
2,Home Mortgage,751412.00000,8 years,0.00000,11.00000,35.00000,1182434.00000,0.00000,nan,0.00000,debt consolidation,Short Term,99999999.00000,308389.00000,13651.00000,742.00000,0
3,Own Home,805068.00000,6 years,0.00000,8.00000,22.50000,147400.00000,1.00000,nan,1.00000,debt consolidation,Short Term,121396.00000,95855.00000,11338.00000,694.00000,0
4,Rent,776264.00000,8 years,0.00000,13.00000,13.60000,385836.00000,1.00000,nan,0.00000,debt consolidation,Short Term,125840.00000,93309.00000,7180.00000,719.00000,0


In [1406]:
df_1.isna().sum()

Home Ownership                     0
Annual Income                   1557
Years in current job             371
Tax Liens                          0
Number of Open Accounts            0
Years of Credit History            0
Maximum Open Credit                0
Number of Credit Problems          0
Months since last delinquent    4081
Bankruptcies                      14
Purpose                            0
Term                               0
Current Loan Amount                0
Current Credit Balance             0
Monthly Debt                       0
Credit Score                    1557
Credit Default                     0
dtype: int64

In [1407]:
df_1['Annual Income'].describe()

count       5943.00000
mean     1366391.72017
std       845339.19963
min       164597.00000
25%       844341.00000
50%      1168386.00000
75%      1640137.00000
max     10149344.00000
Name: Annual Income, dtype: float64

In [1408]:
df_1.loc[(df_1['Annual Income'] > df_1['Annual Income'].quantile(.975)), 'Annual Income'] = df_1['Annual Income'].quantile(.975)
df_1.loc[(df_1['Annual Income'] < df_1['Annual Income'].quantile(.01), 'Annual Income')] = df_1['Annual Income'].quantile(.01)
df_1.loc[(df_1['Annual Income'].isna(), 'Annual Income')] = df_1['Annual Income'].median()

In [1409]:
df_1.head(15)

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.00000,NaN,0.00000,11.00000,26.30000,685960.00000,1.00000,nan,1.00000,debt consolidation,Short Term,99999999.00000,47386.00000,7914.00000,749.00000,0
1,Own Home,1025487.00000,10+ years,0.00000,15.00000,15.30000,1181730.00000,0.00000,nan,0.00000,debt consolidation,Long Term,264968.00000,394972.00000,18373.00000,737.00000,1
2,Home Mortgage,751412.00000,8 years,0.00000,11.00000,35.00000,1182434.00000,0.00000,nan,0.00000,debt consolidation,Short Term,99999999.00000,308389.00000,13651.00000,742.00000,0
3,Own Home,805068.00000,6 years,0.00000,8.00000,22.50000,147400.00000,1.00000,nan,1.00000,debt consolidation,Short Term,121396.00000,95855.00000,11338.00000,694.00000,0
4,Rent,776264.00000,8 years,0.00000,13.00000,13.60000,385836.00000,1.00000,nan,0.00000,debt consolidation,Short Term,125840.00000,93309.00000,7180.00000,719.00000,0
5,Rent,1168386.00000,7 years,0.00000,12.00000,14.60000,366784.00000,0.00000,nan,0.00000,other,Long Term,337304.00000,165680.00000,18692.00000,nan,1
6,Home Mortgage,1511108.00000,10+ years,0.00000,9.00000,20.30000,388124.00000,0.00000,73.00000,0.00000,home improvements,Short Term,99999999.00000,51623.00000,2317.00000,745.00000,0
7,Rent,1040060.00000,10+ years,0.00000,13.00000,12.00000,330374.00000,0.00000,18.00000,0.00000,other,Short Term,250888.00000,89015.00000,19761.00000,705.00000,1
8,Home Mortgage,1168386.00000,5 years,0.00000,17.00000,15.70000,0.00000,1.00000,nan,1.00000,home improvements,Short Term,129734.00000,19.00000,17.00000,nan,0
9,Home Mortgage,1168386.00000,1 year,0.00000,10.00000,24.60000,511302.00000,0.00000,6.00000,0.00000,debt consolidation,Long Term,572880.00000,205333.00000,17613.00000,nan,1


In [1410]:

df_1.loc[(df_1['Years in current job'].isna()), 'Years in current job'] = '10+ years'

In [1411]:
df_1['Years in current job'].mode()

0    10+ years
dtype: object

In [1412]:
df_1['Years in current job'].isna().sum()

0

In [1413]:
df_1.dtypes

Home Ownership                   object
Annual Income                   float64
Years in current job             object
Tax Liens                       float64
Number of Open Accounts         float64
Years of Credit History         float64
Maximum Open Credit             float64
Number of Credit Problems       float64
Months since last delinquent    float64
Bankruptcies                    float64
Purpose                          object
Term                             object
Current Loan Amount             float64
Current Credit Balance          float64
Monthly Debt                    float64
Credit Score                    float64
Credit Default                    int64
dtype: object

In [1414]:
df_1['Number of Open Accounts'].describe()

count   7500.00000
mean      11.13093
std        4.90892
min        2.00000
25%        8.00000
50%       10.00000
75%       14.00000
max       43.00000
Name: Number of Open Accounts, dtype: float64

In [1415]:
df_1['Years of Credit History'].describe()

count   7500.00000
mean      18.31747
std        7.04195
min        4.00000
25%       13.50000
50%       17.00000
75%       21.80000
max       57.70000
Name: Years of Credit History, dtype: float64

In [1416]:
df_1['Maximum Open Credit'].describe()

count         7500.00000
mean        945153.72747
std       16026216.67085
min              0.00000
25%         279229.50000
50%         478159.00000
75%         793501.50000
max     1304726170.00000
Name: Maximum Open Credit, dtype: float64

In [1417]:
df_1.loc[(df_1['Maximum Open Credit'] > df_1['Maximum Open Credit'].quantile(.975)), 'Maximum Open Credit'] = df_1['Maximum Open Credit'].quantile(.975)
df_1.loc[(df_1['Maximum Open Credit'] < df_1['Maximum Open Credit'].quantile(.01)), 'Maximum Open Credit'] = df_1['Maximum Open Credit'].quantile(.01)

In [1418]:
df_1['Maximum Open Credit'].describe()

count      7500.00000
mean     614769.75545
std      482821.25508
min       17269.78000
25%      279229.50000
50%      478159.00000
75%      793501.50000
max     2236204.30000
Name: Maximum Open Credit, dtype: float64

In [1419]:
df_1['Number of Credit Problems'].value_counts()

0.00000    6469
1.00000     882
2.00000      93
3.00000      35
4.00000       9
5.00000       7
6.00000       4
7.00000       1
Name: Number of Credit Problems, dtype: int64

In [1420]:
df_1['Bankruptcies'].value_counts()

0.00000    6660
1.00000     786
2.00000      31
3.00000       7
4.00000       2
Name: Bankruptcies, dtype: int64

In [1421]:
df_1.loc[(df_1['Bankruptcies'].isna()), 'Bankruptcies'] = df_1['Bankruptcies'].quantile(.025)

In [1422]:
df_1['Bankruptcies'].value_counts()

0.00000    6674
1.00000     786
2.00000      31
3.00000       7
4.00000       2
Name: Bankruptcies, dtype: int64

In [1423]:
df_1['Current Loan Amount'].describe()

count       7500.00000
mean    11873177.44507
std     31926122.97473
min        11242.00000
25%       180169.00000
50%       309573.00000
75%       519882.00000
max     99999999.00000
Name: Current Loan Amount, dtype: float64

In [1424]:
df_1.loc[(df_1['Current Loan Amount'] > df_1['Current Loan Amount'].quantile(.75)), 'Current Loan Amount'] = df_1['Current Loan Amount'].quantile(.75)

In [1425]:
df_1['Current Loan Amount'].describe()

count     7500.00000
mean    317466.50907
std     157118.68455
min      11242.00000
25%     180169.00000
50%     309573.00000
75%     519750.00000
max     519882.00000
Name: Current Loan Amount, dtype: float64

In [1426]:
df_1['Current Credit Balance'].describe()

count      7500.00000
mean     289833.23520
std      317871.38238
min           0.00000
25%      114256.50000
50%      209323.00000
75%      360406.25000
max     6506797.00000
Name: Current Credit Balance, dtype: float64

In [1427]:
df_1.loc[(df_1['Current Credit Balance'] > df_1['Current Credit Balance'].quantile(.99)), 'Current Credit Balance'] = df_1['Current Credit Balance'].quantile(.99)
df_1.loc[(df_1['Current Credit Balance'] < df_1['Current Credit Balance'].quantile(.01)), 'Current Credit Balance'] = df_1['Current Credit Balance'].quantile(.01)


In [1428]:
df_1['Monthly Debt'].describe()

count     7500.00000
mean     18314.45413
std      11926.76467
min          0.00000
25%      10067.50000
50%      16076.50000
75%      23818.00000
max     136679.00000
Name: Monthly Debt, dtype: float64

In [1429]:
df_1.loc[(df_1['Credit Score'].isna()), 'Credit Score'] = df_1['Credit Score'].median()


In [1430]:
df_1.loc[(df_1['Credit Score'] > df_1['Credit Score'].quantile(.98)), 'Credit Score'] = df_1['Credit Score'].quantile(.98)

In [1431]:
df_1['Credit Score'].describe()

count   7500.00000
mean    1062.11200
std     1430.59316
min      585.00000
25%      718.00000
50%      731.00000
75%      740.00000
max     7290.00000
Name: Credit Score, dtype: float64

In [1432]:
df_1['Purpose'].value_counts()

debt consolidation      5944
other                    665
home improvements        412
business loan            129
buy a car                 96
medical bills             71
major purchase            40
take a trip               37
buy house                 34
small business            26
wedding                   15
moving                    11
educational expenses      10
vacation                   8
renewable energy           2
Name: Purpose, dtype: int64

In [1433]:
df_1['ratio'] = df_1['Annual Income'] / df_1['Monthly Debt']

In [1434]:
CAT_FEATURE_NAMES = ['Home Ownership','Years in current job', 'Term','Purpose']

In [1435]:
NUM_FEATURE_NAMES = ['Annual Income', 'Tax Liens', 'Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit',
                    'Number of Credit Problems', 'Bankruptcies', 'Current Loan Amount', 'Current Credit Balance', 'Monthly Debt',
                    'Credit Score']

In [1436]:
df_1['ratio'] = df_1['Annual Income'] / df_1['Monthly Debt']

In [1437]:
scaler = StandardScaler()

df_norm = df_1.copy()
df_norm[NUM_FEATURE_NAMES] = scaler.fit_transform(df_norm[NUM_FEATURE_NAMES])

df_1 = df_norm.copy()

In [1438]:
df_1.head(5)

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default,ratio
0,Own Home,-1.33579,10+ years,-0.11095,-0.02667,1.13364,0.14746,1.66478,nan,2.54573,debt consolidation,Short Term,1.28838,-0.91593,-0.87208,-0.21888,0,60.91572
1,Own Home,-0.44638,10+ years,-0.11095,0.78822,-0.42853,1.17434,-0.34098,nan,-0.33710,debt consolidation,Long Term,-0.33416,0.44514,0.00491,-0.22727,1,55.81489
2,Home Mortgage,-0.89498,8 years,-0.11095,-0.02667,2.36918,1.17580,-0.34098,nan,-0.33710,debt consolidation,Short Term,1.28838,0.10610,-0.39103,-0.22378,0,55.04447
3,Own Home,-0.80715,6 years,-0.11095,-0.63785,0.59399,-0.96806,1.66478,nan,2.54573,debt consolidation,Short Term,-1.24800,-0.72614,-0.58498,-0.25733,0,71.00617
4,Rent,-0.85430,8 years,-0.11095,0.38077,-0.66995,-0.47419,1.66478,nan,-0.33710,debt consolidation,Short Term,-1.21971,-0.73611,-0.93363,-0.23985,0,108.11476


In [1439]:
df_1.dtypes

Home Ownership                   object
Annual Income                   float64
Years in current job             object
Tax Liens                       float64
Number of Open Accounts         float64
Years of Credit History         float64
Maximum Open Credit             float64
Number of Credit Problems       float64
Months since last delinquent    float64
Bankruptcies                    float64
Purpose                          object
Term                             object
Current Loan Amount             float64
Current Credit Balance          float64
Monthly Debt                    float64
Credit Score                    float64
Credit Default                    int64
ratio                           float64
dtype: object

In [1440]:
for colname in CAT_FEATURE_NAMES:
    df_1[colname] = pd.Categorical(df_1[colname])
    
df_1[CAT_FEATURE_NAMES].dtypes

Home Ownership          category
Years in current job    category
Term                    category
Purpose                 category
dtype: object

In [1441]:
df_1.dtypes

Home Ownership                  category
Annual Income                    float64
Years in current job            category
Tax Liens                        float64
Number of Open Accounts          float64
Years of Credit History          float64
Maximum Open Credit              float64
Number of Credit Problems        float64
Months since last delinquent     float64
Bankruptcies                     float64
Purpose                         category
Term                            category
Current Loan Amount              float64
Current Credit Balance           float64
Monthly Debt                     float64
Credit Score                     float64
Credit Default                     int64
ratio                            float64
dtype: object

In [1442]:
df_1.dtypes

Home Ownership                  category
Annual Income                    float64
Years in current job            category
Tax Liens                        float64
Number of Open Accounts          float64
Years of Credit History          float64
Maximum Open Credit              float64
Number of Credit Problems        float64
Months since last delinquent     float64
Bankruptcies                     float64
Purpose                         category
Term                            category
Current Loan Amount              float64
Current Credit Balance           float64
Monthly Debt                     float64
Credit Score                     float64
Credit Default                     int64
ratio                            float64
dtype: object

In [1443]:
df_1.isna().sum()

Home Ownership                     0
Annual Income                      0
Years in current job               0
Tax Liens                          0
Number of Open Accounts            0
Years of Credit History            0
Maximum Open Credit                0
Number of Credit Problems          0
Months since last delinquent    4081
Bankruptcies                       0
Purpose                            0
Term                               0
Current Loan Amount                0
Current Credit Balance             0
Monthly Debt                       0
Credit Score                       0
Credit Default                     0
ratio                              0
dtype: int64

In [1444]:
X = df_1.drop(columns = 'Credit Default' )
y = df_1['Credit Default']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.33, shuffle = True, random_state = 42)

In [1445]:
 model_cat = cat.CatBoostClassifier(n_estimators=500, max_depth=3,
                                      silent=True, random_state=42)
    



In [1446]:
df_1['ratio'] = df_1['Annual Income'] / df_1['Monthly Debt']

In [1447]:
df_1.dtypes

Home Ownership                  category
Annual Income                    float64
Years in current job            category
Tax Liens                        float64
Number of Open Accounts          float64
Years of Credit History          float64
Maximum Open Credit              float64
Number of Credit Problems        float64
Months since last delinquent     float64
Bankruptcies                     float64
Purpose                         category
Term                            category
Current Loan Amount              float64
Current Credit Balance           float64
Monthly Debt                     float64
Credit Score                     float64
Credit Default                     int64
ratio                            float64
dtype: object

In [1448]:
X_train

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,ratio
3151,Home Mortgage,-0.21249,10+ years,-0.11095,-0.43412,-0.25811,1.36122,-0.34098,nan,-0.33710,debt consolidation,Short Term,1.10368,1.80204,0.82203,-0.23147,41.55296
4019,Home Mortgage,0.21265,10+ years,-0.11095,0.99195,0.16794,0.47942,-0.34098,nan,-0.33710,debt consolidation,Short Term,-0.30153,0.81595,0.84937,-0.24125,50.20866
4194,Home Mortgage,-0.89908,10+ years,-0.11095,-1.04530,0.79281,-0.54122,-0.34098,nan,-0.33710,debt consolidation,Short Term,-0.91682,-0.48590,-0.75075,-0.22028,80.00256
1825,Own Home,1.06272,3 years,-0.11095,-0.02667,-1.26642,-0.06389,-0.34098,nan,-0.33710,buy a car,Long Term,1.28838,-0.02522,0.50558,-0.22308,79.99918
7363,Home Mortgage,-0.09942,10+ years,-0.11095,0.99195,0.65079,-0.51570,1.66478,68.00000,2.54573,debt consolidation,Short Term,-0.33696,-0.37036,0.02939,-0.22028,66.29896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,Rent,-0.21249,8 years,-0.11095,-1.24902,-0.89718,-0.54596,-0.34098,nan,-0.33710,buy house,Short Term,-1.02017,-0.81289,-1.19575,-0.23147,288.20572
5226,Rent,3.51247,5 years,-0.11095,-0.23040,0.94902,0.07336,-0.34098,34.00000,-0.33710,debt consolidation,Short Term,-0.33500,-0.38464,0.78078,-0.21958,124.67290
5390,Rent,-0.91053,2 years,-0.11095,-0.43412,-1.28062,0.13164,-0.34098,nan,-0.33710,debt consolidation,Short Term,-0.24356,0.14873,-0.26559,-0.22657,48.98079
860,Rent,3.51247,3 years,-0.11095,-0.63785,-0.75516,-0.54947,-0.34098,18.00000,-0.33710,debt consolidation,Short Term,0.97261,-0.00468,-0.01857,-0.22378,190.36166


In [1449]:
model_cat.fit(X_train, y_train, cat_features = CAT_FEATURE_NAMES)

In [1450]:
def classification_report(y_train, y_train_pred, y_valid, y_valid_pred):
    print('TRAIN\n\n' + report(y_train, y_train_pred))
    print('TEST\n\n' + report(y_valid, y_valid_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_valid, y_valid_pred))

In [1451]:
y_train_pred = model_cat.predict(X_train)
y_valid_pred= model_cat.predict(X_valid)



In [1452]:
classification_report(y_train, y_train_pred, y_valid, y_valid_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.79      0.99      0.88      3631
           1       0.91      0.31      0.46      1394

    accuracy                           0.80      5025
   macro avg       0.85      0.65      0.67      5025
weighted avg       0.82      0.80      0.76      5025

TEST

              precision    recall  f1-score   support

           0       0.77      0.98      0.86      1756
           1       0.82      0.28      0.42       719

    accuracy                           0.77      2475
   macro avg       0.80      0.63      0.64      2475
weighted avg       0.78      0.77      0.73      2475

CONFUSION MATRIX

col_0              0    1
Credit Default           
0               1713   43
1                518  201
